# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

**Reminder about our [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating)**

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    pre=len(gold.intersection(pred))/len(pred)
    re=len(gold.intersection(pred))/len(gold)
    print(f'precision: {pre} recall: {re} F1 score: {2*(pre*re)/(pre+re)}')
    pass

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

precision: 0.6 recall: 1.0 F1 score: 0.7499999999999999


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
    
        yield (row[1],row[3],row[4])

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [8]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [9]:
df_dev.head()

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,2,3,Leicestershire_County_Cricket_Club
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club


In [10]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data
def predict(df):
    return_list = []
    for row in df.itertuples():
        sentence_id = row[1]
        sentence = nlp(row[2])
        return_list+=[(sentence_id, e.start, e.end) for e in sentence.ents]
    return set(return_list)

In [11]:
evaluation_report(spans_dev_gold, predict(df_dev))

precision: 0.5213954072029113 recall: 0.702213959776914 F1 score: 0.5984444764511019


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [12]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [13]:
# TODO: Write code here to do your analysis
error_report(df_train[0:500],set(gold_spans(df_train[0:500])),predict(df_train[0:500]))

Sentence: BRUSSELS 1996-08-22
  FP: 1996-08-22
Sentence: The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
  FP: The European Commission
  FP: Thursday
  FN: European Commission
Sentence: Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
  FP: Wednesday
  FP: the European Union 's
  FN: European Union
Sentence: He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .
  FP: the European Union
  FN: European Union
Sentence: He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary mov

*TODO: Write a short text that summarises the errors that you observed*

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. To filter out specific labels it is useful to know the named entity label scheme, which can be found in the [model's documentation](https://spacy.io/models/en#en_core_web_sm). You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [106]:
# TODO: Write code here to improve the span prediction from Problem 2
def predict(df):
    return_list = []
    for row in df.itertuples():
        sentence_id = row[1]
        sentence = nlp(row[2])
        return_list+=[(sentence_id, e.start, e.end) for e in sentence.ents if e.label_ not in ['DATE','MONEY','CARDINAL','PERCENT','ORDINAL','QUANTITY']]
    return set(return_list)

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

In [107]:
evaluation_report(spans_dev_gold, predict(df_dev))

precision: 0.8313301282051282 recall: 0.701368936961298 F1 score: 0.7608396736639472


Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [108]:
# TODO: Write code here to store the predicted spans in a new data frame
df_dev_predict = predict(df_dev)
df_dev_pre = pd.DataFrame(columns=df_dev.columns)
df_dev_pre['sentence_id'],df_dev_pre['beg'],df_dev_pre['end']= [x[0] for x in df_dev_predict],[x[1] for x in df_dev_predict],[x[2] for x in df_dev_predict]
df_dev_pre['sentence'] = [dict(zip(df_dev['sentence_id'].values,df_dev['sentence'].values))[x] for x in df_dev_pre['sentence_id']]
df_dev_pre['label'] = '--NME--'
df_dev_pre

,sentence_id,sentence,beg,end,label
0,0966-152,1. Tanja Damaske ( Germany ) 66.60 metres,4,5,--NME--
1,0995-001,Yuri Kushko,0,2,--NME--
2,1146-002,Iraq 's Deputy Prime Minister Tareq Aziz said ...,10,11,--NME--
3,0967-005,Roberto Bisconti will be sidelined for six Eur...,21,22,--NME--
4,1076-042,Zeleznik ( B ) 4 1 0 3 4 7 3,0,1,--NME--
...,...,...,...,...,...
4987,1118-001,"STIRLING , Scotland 1996-08-31",0,1,--NME--
4988,1152-007,Clinton said on Saturday he had ordered U.S. f...,11,12,--NME--
4989,1009-015,The California Avocado Commission -- which has...,35,36,--NME--
4990,1030-038,December cotton closed 0.95 cent higher at 77....,12,17,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [109]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row[1],row[3],row[4],row[5])


A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

**Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.**

In [110]:
# TODO: Write code here to implement the baseline
def predict_link(df):
    return_list = []
    for row in df.itertuples():
        return_list+=[(row[1],row[3],row[4],'_'.join([x.text for x in nlp(row[2])[row[3]:row[4]]]))]
    return set(return_list)

evaluation_report(set(gold_mentions(df_dev)), predict_link(df_dev_pre))

precision: 0.30809294871794873 recall: 0.25992901808348823 F1 score: 0.28196901640847005


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [111]:
with bz2.open('kb.tsv.bz2', 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [112]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [113]:
# TODO: Write code here to implement the "most probable entity" method.
def entity_link(df):
    return_list = []
    for row in df.itertuples():
        entity = df_kb.loc[df_kb.mention == nlp(row[2])[row[3]:row[4]].text][0:1]['entity'].values
        if len(entity)==0:
            return_list+=[(row[1],row[3],row[4],'--NME--')]
        else:
            return_list+=[(row[1],row[3],row[4],entity[0])]
    return set(return_list)


In [114]:
evaluation_report(set(gold_mentions(df_dev)), entity_link(df_dev_pre))

precision: 0.6374198717948718 recall: 0.5377725198580362 F1 score: 0.5833715280960676


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [115]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [116]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [117]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [118]:
# TODO: Write code here to implement the context-sensitive disambiguation method
import math
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

vectorizer1 = CountVectorizer()
vectorizer1.fit(np.append(df_contexts.context.values,df_contexts.mention.values))

dict_mention = dict()

for m in set(df_contexts.mention):
    contexts=df_contexts[df_contexts.mention==m]['context'].values
    entity=df_contexts[df_contexts.mention==m]['entity'].values
    #complete contexts by replacing the '@' 
    contexts_replace = [x[0].replace('@',x[1]) for x in zip(contexts,entity)]
    #compute prior probabilities
    prior_prob_list=[df_kb.prob[(df_kb.mention == m)&(df_kb.entity==x)].values[0] for x in sorted(set(entity))]
    #vectorize context as input data
    X=vectorizer1.transform(contexts_replace)
    model = MultinomialNB(class_prior=prior_prob_list)
    dict_mention[m]=model.fit(X,entity)


In [204]:
# prediction
def predict_cs(df):
    return_list = []
    for row in df.itertuples():
        # extract mentions from sentences
        mention =nlp(row[2])[row[3]:row[4]].text
        if mention in dict_mention.keys():
            context = [nlp(row[2])[row[3]-5:row[4]-5].text]
            #predict using the model stored in dictionary
            entity_pre = dict_mention[mention].predict(vectorizer1.transform(context))[0]
            return_list+=[(row[1],row[3],row[4],entity_pre)]
        
        else:
            # for the mentions which don't have contexts, use entity_link instead
            return_list+=[x for x in entity_link(pd.DataFrame([row[1:]]))]

    return set(return_list)


In [203]:
evaluation_report(set(gold_mentions(df_dev)), predict_cs(df_dev_pre))

precision: 0.6396233974358975 recall: 0.5396315700523914 F1 score: 0.5853882115684297


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 5.1:** In Problem&nbsp;3, you did an error analysis on the task of recognizing text spans mentioning named entities. Summarize your results. Pick one type of error that you observed. How could you improve the model&rsquo;s performance on this type of error? What resources (such as domain knowledge, data, compute) would you need to implement this improvement?

**RQ 5.2:** Thinking back about Problem&nbsp;6, explain what the word *context* refers to in the task addressed there, and how context can help to disambiguate between different entities. Suggest other types of context that you could use for disambiguation.

**RQ 5.3:** One type of entity mentions that we did not cover explicitly in this lab are pronouns. As an example, consider the sentence pair *Ruth Bader Ginsburg was an American jurist*. *She served as an associate justice of the Supreme Court from 1993 until her death in 2020*. What facts would you want to extract from this sentence pair? How do pronouns make fact extraction hard?

*TODO: Enter your answers here*

**This was the last lab in the Text Mining course. Congratulations! 🥳**